In [22]:
import torch 
import pyro
import pyro.infer
import pyro.optim
import matplotlib.pyplot as plt
from torch.distributions import constraints
import numpy as np
pyro.set_rng_seed(101)

In [3]:
### pyro.distributions is a wrapper around toch.distributions

## We can create torch.distributions objects 
mean = 0
var = 1
normal = torch.distributions.Normal(mean,var)

###  These objects have the following methods 

print(normal.rsample())
print(normal.log_prob(normal.rsample()))


tensor(-1.3905)
tensor(-1.2512)


Why do we use Torch distirbutions -> fast tensor math and autograd capabilities


In [20]:
### Doing inference in the beta-binomial model using SVI in pyro 

## Write the generative process as a function

def beta(n,a,b):
    theta = pyro.sample('theta',pyro.distributions.Beta(torch.tensor(a),torch.tensor(b)))
    k = pyro.sample('k',pyro.distributions.Binomial(torch.tensor(n,dtype=torch.float64),theta))
    return k

## Condition the model 
obs = {'k':5}
conditioned_beta = pyro.condition(beta,data = obs)

def guide_beta(n,a,b):
    a_post = pyro.param('a_post',torch.tensor(a,dtype=torch.float64),constraint = constraints.positive)
    b_post = pyro.param('b_post',torch.tensor(b,dtype=torch.float64),constraint = constraints.positive)
    theta = pyro.sample('theta',pyro.distributions.Beta(a_post,b_post))
    return theta 

### Doing inference on theta 

### Setting param 

n = 10 
a = 2
b = 3
k = 5
## conditioning the model 

obs = {'k':k}
conditioned_beta = pyro.condition(beta,data = obs)

pyro.clear_param_store()
svi = pyro.infer.SVI(model = conditioned_beta,guide = guide_beta,optim=pyro.optim.SGD({'lr':0.001,'momentum':0.1}),loss=pyro.infer.Trace_ELBO())

losses, a_post, b_post = [], [],[]
iterations = 1000
for _ in range(iterations):
    losses.append(svi.step(n,a,b))
    a_post.append(pyro.param('a_post').item())
    b_post.append(pyro.param('b_post').item())

plt.clf()
plt.subplot(2,2,1)
plt.plot(losses,'b-')
plt.subplot(2,2,2)
plt.plot(a_post,'b-')
plt.hlines(a + k)
plt.subplot(2,2,3)
plt.plot(b_post,'b-')
plt.hlines(b + n-k)
    
    

RuntimeError: expected backend CPU and dtype Double but got backend CPU and dtype Long

In [28]:
### Trying to implement Likelihood Weighting for Pyro to get mean and variance estimates of the posterior distribution

## Beta-Bernoulli model 
def conditioned_model(n,theta,obs):
    logW = 0
    logW += pyro.distributions.Binomial(n,theta).log_prob(torch.tensor(obs,dtype=torch.float64))
    return logW


def guide(a,b):
    theta = pyro.distributions.Beta(a,b).rsample()
    return theta
    

def ImportanceSampling(r,conditioned_model,guide,L,a,b,n,obs):
    '''
    Guide must return all parameters
    L is the number of samples 
    '''
    estimate = 0
    weight_sum = 0
    for i in range(L):
        theta_sample = guide(a,b)
        weight = np.exp(conditioned_model(n,theta_sample,obs))
        weight_sum += weight
        estimate += weight * r(theta_sample)
    return estimate/weight_sum
        

ImportanceSampling(lambda x:x**2,conditioned_model,guide,100,2,5,10,3)
    
    
    

tensor(0.0996, dtype=torch.float64)

In [34]:
### Write a general function that implements importance sampling for a generative process where we observed some values 

### What properties must model suffice 

def model(observations,r):
    unobserved = []
    LW = 0
    
    return 

def ImportanceSampling(model,observations,L):
    '''
    Input:
    
    observations is a dictionary that tells us which sample statements are observed
    model is just a generative model
    L is the number of samples used for importance sampling
    
    Output:
    
    Returns a list of estimates of the values of sample statments that were not observed. 
    '''
    for _ in range(L):
        likelihood_weight,r = model(observations)
        estimate 
        
    


0.01156910999999998

In [33]:
def var_beta(p,q):
    return (p*q)/((p+q+1)*(p+q)**2)

var_beta(5,12)

0.011534025374855825

In [18]:
a = 3
b = 4
pyro.distributions.Beta(a,b).rsample()
n = 10 
p = 0.3
k = torch.tensor(4,dtype=torch.float64)
torch.distributions.Binomial(n,p).log_prob(k)



tensor(-1.6088, dtype=torch.float64)

In [16]:
torch.lgamma(torch.tensor(5,dtype=torch.float64))

tensor(3.1781, dtype=torch.float64)

In [8]:
with Student('Niklas') as stud:
    print(stud.name)
    print(stud.values())

Niklas


AttributeError: 'Student' object has no attribute 'values'

In [15]:
from collections import OrderedDict
PYRO_STACK = []

class Messenger(object):
    def __init__(self, fn=None):
        self.fn = fn

    # Effect handlers push themselves onto the PYRO_STACK.
    # Handlers earlier in the PYRO_STACK are applied first.
    def __enter__(self):
        PYRO_STACK.append(self)

    def __exit__(self, *args, **kwargs):
        assert PYRO_STACK[-1] is self
        PYRO_STACK.pop()

    def process_message(self, msg):
        pass

    def postprocess_message(self, msg):
        pass

    def __call__(self, *args, **kwargs):
        with self:
            return self.fn(*args, **kwargs)

In [16]:
class trace(Messenger):
    def __enter__(self):
        super(trace, self).__enter__()
        self.trace = OrderedDict()
        return self.trace

    # trace illustrates why we need postprocess_message in addition to process_message:
    # We only want to record a value after all other effects have been applied
    def postprocess_message(self, msg):
        assert msg["type"] != "sample" or msg["name"] not in self.trace, \
            "sample sites must have unique names"
        self.trace[msg["name"]] = msg.copy()

    def get_trace(self, *args, **kwargs):
        self(*args, **kwargs)
        return self.trace

In [17]:
with trace() as param_capture:
    print(param_capture.values())

odict_values([])


TypeError: 'type' object does not support item assignment

In [27]:
def my_printer(string):
    print(string)

def function_prodcution():
    return my_printer 

for elem in function_prodcution():
    elem('it works ')
    
    

TypeError: 'function' object is not iterable

In [30]:
float('+-0.4545')


ValueError: could not convert string to float: '+-0.4545'

In [ ]:
fl